# Classification Model

# PadChest Dataset

In [1]:
from fastai.basics import *
from fastai.text.all import *
import warnings
warnings.filterwarnings('ignore')
# Read in the train and test sets.
path = Path('/home/jupyter/data/padchest')
df_lm = pd.read_csv(path/"lm.csv")
df_cl = pd.read_csv(path/"labels.csv")
df_train = pd.read_csv(path/"train.csv")
df_test = pd.read_csv(path/"test.csv")

# Multi-Label Classifier

In [2]:
# fix result 
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
SEED = 42
seed_everything(SEED)

### 1. Data Block

In [3]:
labels = ["Atelectasis", "Cardiomegaly", "Consolidation",
          "Edema", "Fracture", "No Finding", "Pleural Effusion", 
          "Pneumonia", "Pneumothorax"]

In [4]:
bs_cl = (TextBlock.from_df('Report'), 
          MultiCategoryBlock(encoded=True, vocab=labels))

In [5]:
db_cl = DataBlock(blocks=bs_cl,
                      get_x=ColReader('text'),
                      get_y=ColReader(labels),
                      splitter=ColSplitter('is_valid'))

In [6]:
db_cl.splitter(df_cl)

((#22275) [7090,7091,7092,7093,7094,7095,7096,7097,7098,7099...],
 (#7090) [0,1,2,3,4,5,6,7,8,9...])

### 2. Data Loader

In [8]:
dl_cl = db_cl.dataloaders(df_cl)

In [9]:
dl_cl.show_batch()

,text,None
0,xxbos indic cardiotorac limit alto normal . con respect estudi previ xxunk xxunk xxunk xxunk observ xxunk derram pleural derech . pers xxunk xxunk xxunk derech . xxunk estudi xxunk torac observ xxunk xxunk xxunk xxunk xxunk xxunk densid xxunk xxunk xxunk xxunk xxunk xxunk atelectasi xxunk xxunk xxunk . xxunk xxunk xxunk probabl proces inflamatori xxunk xxunk xxunk sin xxunk descart xxunk xxunk xxunk . xxunk xxunk xxunk derram pleural derech . xxunk xxunk xxunk derech xxunk xxunk xxunk asoci atelectasi xxunk xxunk . xxunk xxunk etiologi inflamatori sin xxunk descart xxunk xxunk xxunk . valor tratamient xxunk xxunk xxunk xxunk xxunk . xxunk xxunk hallazg xxunk xxunk xxunk xxunk xxunk derech xxunk torac pacient xxunk xxunk xxunk .,Atelectasis;Cardiomegaly;Pleural Effusion
1,xxbos engros xxunk xxunk asoci xxunk derram pleural . infiltr xxunk derech hili derech aument taman probabl etiologi vascul sin xxunk descart etiologi xxunk xxunk . densid nodul xxunk xxunk izquierd . xxunk xxunk xxunk izquierd no xxunk xxunk xxunk xxunk xxunk xxunk mediastin xxunk xxunk pacient no xxunk estudi . apreci xxunk xxunk proyect siluet xxunk xxunk xxunk proces xxunk . xxunk xxunk xxunk con xxunk xxunk pacient xxunk valor si pued xxunk xxunk xxunk . xxunk xxunk estudi con xxunk xxunk .,Pleural Effusion
2,xxbos . . xxunk clinic xxunk . xxunk . . . . . xxunk torax xxunk xxunk . . . . xxunk xxunk . . xxunk xxunk pleural derech xxunk xxunk . . . xxunk xxunk xxunk izquierd xxunk . . xxunk xxunk xxunk superior . . . xxunk xxunk con xxunk 5 xxunk xxunk . . . infiltr interstici alveolar xxunk bilateral xxunk probabl relacion con xxunk xxunk xxunk sin cambi signific respect estudi previ xxunk xxunk xxunk xxunk .,Pneumonia
3,xxbos . . . . xxunk xxunk xxunk xxunk . . . xxunk xxunk con xxunk xxunk xxunk . . . xxunk xxunk xxunk con xxunk xxunk xxunk xxunk derech xxunk xxunk xxunk . . . xxunk xxunk pleural nivel basal derech . . . xxunk xxunk pleural nivel basal izquierd pers xxunk xxunk xxunk pleural izquierd con xxunk xxunk xxunk xxunk con xxunk mejori radiolog respect xxunk xxunk xxunk . . . atelectasi basal izquierd xxunk xxunk sin cambi .,Atelectasis;Pneumothorax
4,xxbos sign epoc con xxunk xxunk xxunk retrocardiac . pequen atelectasi xxunk sen costofren derech . aort xxunk . aument xxunk dorsal con xxunk anterior relacion con xxunk xxunk xxunk . xxunk . xxunk . xxunk xxunk xxunk xxunk xxunk anterior xxunk xxunk xxunk xxunk anterior xxunk xxunk xxunk relacion con xxunk . xxunk xxunk xxunk xxunk xxunk 5 xxunk xxunk xxunk xxunk xxunk xxunk xxunk 5 xxunk xxunk . xxunk aort xxunk xxunk xxunk xxunk . xxunk bilateral .,Atelectasis
5,xxbos xxunk con estudi previ xxunk xxunk xxunk xxunk . xxunk xxunk torac con xxunk proyect camp inferior derech sin cambi . xxunk xxunk xxunk xxunk aument xxunk ambos hemitorax con xxunk xxunk xxunk derram pleural bilateral . xxunk xxunk xxunk xxunk xxunk xxunk xxunk derech . respect xxunk previ xxunk xxunk xxunk xxunk xxunk xxunk xxunk proyect camp pulmon inferior izquierd no visualiz xxunk estudi previ context pacient xxunk valor xxunk xxunk xxunk . xxunk sin cambi .,Pleural Effusion
6,xxbos cambi pulmonar cronic . engros xxunk xxunk . patron intersticial xxunk hemitorax derech . visualiz nivel xxunk nivel xxunk xxunk xxunk aument densid xxunk xxunk xxunk taman relacion primer xxunk xxunk descart xxunk sin xxunk descart xxunk infecci si xxunk con clinic . aument xxunk xxunk izquierd si xxunk xxunk relacion con elongacion aort no pued descart xxunk xxunk . xxunk estudi con tc torax xxunk xxunk . xxunk pacient xxunk xxunk xxunk xxunk tc torax .,Pneumonia
7,xxbos xxunk xxunk con xxunk xxunk xxunk . xxunk xxunk xxunk con xxunk xxunk xxunk izquierd xxunk xxunk xxunk xxunk superior . xxunk xxunk pleural derech con xxunk xxunk pulmon . xxunk con xxunk previ xxunk xxunk xxunk xxunk apreci xxunk radiolog con aument infiltr alveolar xxunk derech xxunk aument derram pleural xxunk hallazg pued relacion con sign insuficient xxunk dere

### 3. Training

In [10]:
text_classifier_learner??

Signature:
text_classifier_learner(
    dls,
    arch,
    seq_len=72,
    config=None,
    backwards=False,
    pretrained=True,
    drop_mult=0.5,
    n_out=None,
    lin_ftrs=None,
    ps=None,
    max_len=1440,
    y_range=None,
    loss_func=None,
    opt_func=<function Adam at 0x7fc05337c0e0>,
    lr=0.001,
    splitter=<function trainable_params at 0x7fc060c063b0>,
    cbs=None,
    metrics=None,
    path=None,
    model_dir='models',
    wd=None,
    wd_bn_bias=False,
    train_bn=True,
    moms=(0.95, 0.85, 0.95),
)
Source:   
@delegates(Learner.__init__)
def text_classifier_learner(dls, arch, seq_len=72, config=None, backwards=False, pretrained=True, drop_mult=0.5, n_out=None,
                            lin_ftrs=None, ps=None, max_len=72*20, y_range=None, **kwargs):
    "Create a `Learner` with a text classifier from `dls` and `arch`."
    vocab = _get_text_vocab(dls)
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferre

In [11]:
loss_func = BCEWithLogitsLossFlat(thresh=0.8)
metrics = [partial(accuracy_multi, thresh=0.8), 
           F1ScoreMulti(average='macro'),
           PrecisionMulti (average='macro'),
           RecallMulti (average='macro'),
           RocAucMulti (average='macro')]

In [12]:
learn_cl = text_classifier_learner(dl_cl, AWD_LSTM, metrics=metrics, loss_func=loss_func)

In [13]:
learn_cl.to_fp16()
learn_cl.fine_tune(10)

epoch,train_loss,valid_loss,accuracy_multi,f1_score,precision_score,recall_score,roc_auc_score,time
0,0.365805,0.291285,0.877903,0.237123,0.382799,0.184396,0.713495,00:10


epoch,train_loss,valid_loss,accuracy_multi,f1_score,precision_score,recall_score,roc_auc_score,time
0,0.300189,0.249865,0.896756,0.308923,0.506357,0.270091,0.796495,00:13
1,0.260593,0.227635,0.908228,0.381185,0.528449,0.342049,0.850376,00:13
2,0.238985,0.201587,0.917035,0.448575,0.553227,0.403259,0.862155,00:13
3,0.206132,0.186601,0.925717,0.481846,0.551464,0.449531,0.888032,00:13
4,0.195556,0.167046,0.933302,0.522988,0.581262,0.484182,0.897201,00:13
5,0.183333,0.149813,0.939994,0.554304,0.588077,0.530022,0.916301,00:13
6,0.183114,0.139998,0.944460,0.574411,0.643349,0.548797,0.919376,00:13
7,0.173082,0.129450,0.949052,0.597441,0.767259,0.570617,0.928937,00:13
8,0.164232,0.124662,0.951011,0.606771,0.767094,0.580429,0.929181,00:13
9,0.156645,0.126738,0.952562,0.600620,0.655313,0.588026,0.929379,00:13
